# k-NN

by Kirill Bykov, Oleg Vlasovets, Ilya

In [5]:
#imports
import pandas as pd #pandas for reading csv file
import numpy as np  #numpy for vectorising
from random import randrange
import seaborn as sns
from tqdm import tqdm
%matplotlib inline

In [6]:
def read_MNIST(file_path):
    df = pd.read_csv(file_path, header = None)
    Y = np.array(df[0])
    X = np.array(df.iloc[:,1:])
    return X, Y

def distance(A, B, metric = "euclidean"):
    if metric == "euclidean":
        return np.linalg.norm(A-B)

def first_most_frequent(arr):
    c = 0
    num = arr[0]
    for i in arr:
        amount_times = arr.count(i)
        if amount_times > c:
            c = amount_times
            num = i
    return num

def vote(NN, Y):
    n = len(NN)
    votes = np.zeros(n).astype(int)
    for i in range(0, n):
        votes[i] = first_most_frequent([Y[j] for j in NN[i]])
    return votes
    
        
def kNN(X_train, Y_train, X_test, k, metric = "euclidean"):
    n = len(X_train)
    n_test = len(X_test)
    D = np.zeros((n_test, n))
    NN = np.zeros((n_test,k)).astype(int)
    for i in range(0,n_test):
        for j in range(0,n):
            D[i, j] = distance(X_test[i], X_train[j], metric)
        NN[i] = D[i].argsort()[:k]
    pred = vote(NN, Y_train)
    return pred

def accuracy(Y_pred, Y_true):
    return np.mean(1 - np.equal(Y_true, Y_pred, dtype=int))

In [8]:
results = []
for k in tqdm(range(1,21)):
    X_train, Y_train = read_MNIST("MNIST_train_small.csv")
    X_test, Y_test = read_MNIST("MNIST_test_small.csv")
    Y_pred = kNN(X_train, Y_train, X_test, k)
    results.append(accuracy(Y_pred, Y_test))
    print(results)

ax = sns.lineplot(x=[i for i in range(1,21)], y=results)

  0%|                                                   | 0/20 [00:00<?, ?it/s]

[0.085]


  5%|██                                        | 1/20 [01:59<37:59, 119.97s/it]

KeyboardInterrupt: 

### Cross-validation

In [10]:
def split_data(dataset, folds=10): #can change k manually as you want
    dataset_split = list()
    dataset_copy = list(dataset)
    fold_size = int(len(dataset) / folds)
    for i in range(folds):
        fold = list()
        while len(fold) < fold_size:
            index = randrange(len(dataset_copy))
            fold.append(dataset_copy.pop(index))
        dataset_split.append(fold)
    return dataset_split

In [8]:
X, Y = read_MNIST("MNIST_train_small.csv")

In [13]:
def cross_validation(k):
    predictions = []
    f = split_data(X)
    for x in range(len(f)):
        neighbors = kNN(X_train, Y_train, X_test, k, metric = "euclidean")
        result = vote(NN, Y)
        predictions.append(result)
    accuracy = accuracy(Y_pred, Y_true)
    print('Accuracy: ' + repr(accuracy) + '%')